In [1]:
import mysql.connector

remote = mysql.connector.connect(
    host = "database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "cwh",
    password = "0000",
    database = "pig"
)

In [2]:
cur = remote.cursor()

In [3]:
# ground_price

import pandas as pd

df = pd.read_csv('../data/ground_price.csv', encoding='EUC-KR')
df

,년도,사업명,지역(도),계약건수,면적(HA),금액(백만원)
0,2023,농지규모화-매도,경기,13,7,1518
1,2023,농지규모화-매도,강원,14,12,2046
2,2023,농지규모화-매도,충북,30,10,1956
3,2023,농지규모화-매도,충남,110,52,10974
4,2023,농지규모화-매도,전북,167,74,15976
5,2023,농지규모화-매도,전남,224,94,18694
6,2023,농지규모화-매도,경북,61,21,3898
7,2023,농지규모화-매도,경남,49,14,2735
8,2023,농지규모화-임대,경기,27,45,1472
9,2023,농지규모화-임대,강원,3,4,119


In [18]:
df['금액/면적(원/m2)'] = (df['금액(백만원)'] / df['면적(HA)'] * 1000000 / 10000).round(0).astype(int)
df[:8]

,년도,사업명,지역(도),계약건수,면적(HA),금액(백만원),금액/면적(원/m2)
0,2023,농지규모화-매도,경기,13,7,1518,21686
1,2023,농지규모화-매도,강원,14,12,2046,17050
2,2023,농지규모화-매도,충북,30,10,1956,19560
3,2023,농지규모화-매도,충남,110,52,10974,21104
4,2023,농지규모화-매도,전북,167,74,15976,21589
5,2023,농지규모화-매도,전남,224,94,18694,19887
6,2023,농지규모화-매도,경북,61,21,3898,18562
7,2023,농지규모화-매도,경남,49,14,2735,19536


In [19]:
cur.execute("DELETE FROM ground_price")
for row in df[:8].itertuples():
    print(row[1], row[3], row[4], row[5], row[6], row[7])
    cur.execute("INSERT INTO ground_price VALUES (%s, %s, %s, %s, %s, %s)", (row[1], row[3], row[4], row[5], row[6], row[7]))

remote.commit()

2023 경기 13 7 1518 21686
2023 강원 14 12 2046 17050
2023 충북 30 10 1956 19560
2023 충남 110 52 10974 21104
2023 전북 167 74 15976 21589
2023 전남 224 94 18694 19887
2023 경북 61 21 3898 18562
2023 경남 49 14 2735 19536


In [21]:
import urllib.request # urllib = URL 처리를 위한 파이썬 패키지
import pandas as pd
import json

breeding_price_df = pd.read_excel('./data/breeding_price.xlsx')
breeding_price_df.head()

/home/wonho/project/eda-repo-4/.venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,비목별,2015,2015.1,2015.2,2015.3,2015.4,2016,2016.1,2016.2,2016.3,...,2022,2022.1,2022.2,2022.3,2022.4,2023,2023.1,2023.2,2023.3,2023.4
0,비목별,"1,000두미만","1,000~1,999두","2,000~2,999두","3,000두이상",평균,"1,000두미만","1,000~1,999두","2,000~2,999두","3,000두이상",...,"1,000두미만","1,000~1,999두","2,000~2,999두","3,000두이상",평균,"1,000두미만","1,000~1,999두","2,000~2,999두","3,000두이상",평균
1,비용합계,355424,323013,300088,296252,307077,370138,318897,316286,282701,...,465382,418036,392127,374547,393089,476096,446943,442953,403280,423855
2,가축비,107895,87626,73170,68329,76176,115825,85682,79870,72543,...,123700,87593,72427,62911,73972,111322,100790,95717,67553,82075
3,사료비,169008,169920,165461,166465,167173,173808,166773,168637,151714,...,233179,228413,232149,230330,230481,232766,236449,239019,239697,238468
4,수도광열비,3277,3940,3634,3410,3530,3638,4153,3713,3734,...,5476,5928,6216,5971,5967,8375,9031,7765,6834,7533


In [22]:
breeding_price_df.columns = breeding_price_df.columns.str.replace(r'\.\d+', '', regex=True)
breeding_price_df = breeding_price_df.transpose()
breeding_price_df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
비목별,비목별,비용합계,가축비,사료비,수도광열비,방역치료비,자동차비,감가상각비,수리유지비,임차료,...,고용노동비,분뇨처리비,생산관리비,기타비용,일반비소계,자가노동비,자본용역비,고정자본이자,유동자본이자,토지용역비
2015,"1,000두미만",355424,107895,169008,3277,7380,2407,1882,344,181,...,4094,6714,1271,1130,324881,23565,5866,2192,3674,1112
2015,"1,000~1,999두",323013,87626,169920,3940,8235,1272,818,101,353,...,7725,7619,1252,1045,305275,11554,5163,1914,3249,1021
2015,"2,000~2,999두",300088,73170,165461,3634,9011,1252,640,168,444,...,9860,7969,1043,1021,287385,6385,5393,1970,3423,925
2015,"3,000두이상",296252,68329,166465,3410,9901,909,601,173,135,...,12111,6589,1538,1433,286718,2737,5976,2500,3476,821


In [23]:
breeding_price_df.columns = breeding_price_df.iloc[0]
breeding_price_df = breeding_price_df[1:]
breeding_price_df.head()

비목별,비목별,비용합계,가축비,사료비,수도광열비,방역치료비,자동차비,감가상각비,수리유지비,임차료,...,고용노동비,분뇨처리비,생산관리비,기타비용,일반비소계,자가노동비,자본용역비,고정자본이자,유동자본이자,토지용역비
2015,"1,000두미만",355424,107895,169008,3277,7380,2407,1882,344,181,...,4094,6714,1271,1130,324881,23565,5866,2192,3674,1112
2015,"1,000~1,999두",323013,87626,169920,3940,8235,1272,818,101,353,...,7725,7619,1252,1045,305275,11554,5163,1914,3249,1021
2015,"2,000~2,999두",300088,73170,165461,3634,9011,1252,640,168,444,...,9860,7969,1043,1021,287385,6385,5393,1970,3423,925
2015,"3,000두이상",296252,68329,166465,3410,9901,909,601,173,135,...,12111,6589,1538,1433,286718,2737,5976,2500,3476,821
2015,평균,307077,76176,167173,3530,9231,1163,762,174,227,...,10237,7003,1385,1271,293658,6792,5727,2283,3444,900


In [38]:
breeding_price_df = breeding_price_df.drop(columns=['　　　유동자본이자'], errors='ignore')
breeding_price_df.head()

비목별,비목별,비용합계,가축비,사료비,수도광열비,방역치료비,자동차비,농구비,영농시설비,기타재료비,차입금이자,토지임차료,고용노동비,분뇨처리비,생산관리비,기타비용,일반비소계,자가노동비,자본용역비,토지용역비
2015,"1,000두미만",355424,107895,169008,3277,7380,2407,8120,7184,2655,2846,900,4094,6714,1271,1130,324881,23565,5866,1112
2015,"1,000~1,999두",323013,87626,169920,3940,8235,1272,5829,6100,1705,2963,44,7725,7619,1252,1045,305275,11554,5163,1021
2015,"2,000~2,999두",300088,73170,165461,3634,9011,1252,5289,5565,1841,2266,3,9860,7969,1043,1021,287385,6385,5393,925
2015,"3,000두이상",296252,68329,166465,3410,9901,909,4295,7203,1630,2896,9,12111,6589,1538,1433,286718,2737,5976,821
2015,평균,307077,76176,167173,3530,9231,1163,5075,6745,1768,2806,95,10237,7003,1385,1271,293658,6792,5727,900


In [40]:
breeding_price_df.columns

Index(['비목별', '비용합계', '가축비', '사료비', '수도광열비', '방역치료비', '자동차비', '농구비', '영농시설비',
       '기타재료비', '차입금이자', '토지임차료', '고용노동비', '분뇨처리비', '생산관리비', '기타비용', '일반비소계',
       '자가노동비', '자본용역비', '토지용역비'],
      dtype='object', name='비목별')

In [ ]:
for year, row in breeding_price_df.iterrows():
    cur.execute(f"""
                    INSERT INTO
                        breeding_price
                    VALUES ({year}, '{row['비목별']}', {row['비용합계']}, {row['가축비']}, {row['사료비']}, {row['수도광열비']}, {row['방역치료비']}, {row['자동차비']}, {row['농구비']}, {row['영농시설비']}, {row['기타재료비']}, {row['차입금이자']}, {row['토지임차료']}, {row['고용노동비']}, {row['분뇨처리비']}, {row['생산관리비']}, {row['기타비용']}, {row['일반비소계']}, {row['자가노동비']}, {row['자본용역비']}, {row['토지용역비']}) 
                """)


In [58]:
remote.commit()

In [69]:
# 운영비 = 수도광열비(전기세) + 농구비(농기구) + 영농시설비(온실) + 기타재료비(비료, 살충제) + 차입금이자 + 토지임차료 + 고용노동비 + 분뇨처리비 + 생산관리비 + 기타비용
breeding_price_latest_df = breeding_price_df[['수도광열비', '농구비', '영농시설비', '기타재료비', '차입금이자', '토지임차료', '고용노동비', '분뇨처리비', '생산관리비', '기타비용']].iloc[-5:]
breeding_price_latest_df

비목별,수도광열비,농구비,영농시설비,기타재료비,차입금이자,토지임차료,고용노동비,분뇨처리비,생산관리비,기타비용
2023,8375,12316,12637,2652,5820,128,11783,11381,2152,2293
2023,9031,10585,9710,1690,6261,373,12289,12520,1624,1467
2023,7765,10875,8443,1077,4352,0,16419,16604,1966,1516
2023,6834,7421,7830,1378,4252,0,18586,11195,2397,3269
2023,7533,8991,8653,1480,4772,81,16498,12405,2155,2546


In [70]:
breeding_price_latest_df['합계'] = breeding_price_latest_df[['수도광열비', '농구비', '영농시설비', '기타재료비', '차입금이자', '토지임차료', '고용노동비', '분뇨처리비', '생산관리비', '기타비용']].sum(axis=1)
breeding_price_latest_df

비목별,수도광열비,농구비,영농시설비,기타재료비,차입금이자,토지임차료,고용노동비,분뇨처리비,생산관리비,기타비용,합계
2023,8375,12316,12637,2652,5820,128,11783,11381,2152,2293,69537
2023,9031,10585,9710,1690,6261,373,12289,12520,1624,1467,65550
2023,7765,10875,8443,1077,4352,0,16419,16604,1966,1516,69017
2023,6834,7421,7830,1378,4252,0,18586,11195,2397,3269,63162
2023,7533,8991,8653,1480,4772,81,16498,12405,2155,2546,65114


In [71]:
breeding_price_latest_df.index = ['1000두 이하', '1000~1999두', '2000~2999두', '3000두 이상', '평균']
breeding_price_latest_df

비목별,수도광열비,농구비,영농시설비,기타재료비,차입금이자,토지임차료,고용노동비,분뇨처리비,생산관리비,기타비용,합계
1000두 이하,8375,12316,12637,2652,5820,128,11783,11381,2152,2293,69537
1000~1999두,9031,10585,9710,1690,6261,373,12289,12520,1624,1467,65550
2000~2999두,7765,10875,8443,1077,4352,0,16419,16604,1966,1516,69017
3000두 이상,6834,7421,7830,1378,4252,0,18586,11195,2397,3269,63162
평균,7533,8991,8653,1480,4772,81,16498,12405,2155,2546,65114


In [72]:
breeding_price_latest_df.columns

Index(['수도광열비', '농구비', '영농시설비', '기타재료비', '차입금이자', '토지임차료', '고용노동비', '분뇨처리비',
       '생산관리비', '기타비용', '합계'],
      dtype='object', name='비목별')

In [75]:
for kind, row in breeding_price_latest_df.iterrows():
    cur.execute(f"""
                    INSERT INTO
                        operating_price
                    VALUES ('{kind}', 
                            {row['수도광열비']}, 
                            {row['농구비']}, 
                            {row['영농시설비']}, 
                            {row['기타재료비']}, 
                            {row['차입금이자']}, 
                            {row['토지임차료']}, 
                            {row['고용노동비']}, 
                            {row['분뇨처리비']}, 
                            {row['생산관리비']}, 
                            {row['기타비용']}, 
                            {row['합계']}) 
                """)
remote.commit()

In [98]:
all_pig_number = 1100

def print_cost(result_list):
    print(f"전기세수도광열비 : {all_pig_number * result_list[0]}원 ")
    print(f"농구비 : {all_pig_number * result_list[1]}원 ")
    print(f"영농시설비 : {all_pig_number * result_list[2]}원 ")
    print(f"기타재료비 : {all_pig_number * result_list[3]}원 ")
    print(f"차입금이자 : {all_pig_number * result_list[4]}원 ")
    print(f"토지임차료 : {all_pig_number * result_list[5]}원 ")
    print(f"고용노동비 : {all_pig_number * result_list[6]}원 ")
    print(f"분뇨처리비 : {all_pig_number * result_list[7]}원 ")
    print(f"생산관리비 : {all_pig_number * result_list[8]}원 ")
    print(f"기타비용 : {all_pig_number * result_list[9]}원 ")
    print(f"------------------------------------")
    print(f"예상되는 총 1년 운영비용 : {all_pig_number * result_list[10]}원 입니다.")
    print(f"예상되는 월 운영비용 : {int(all_pig_number / 12 * result_list[10])}원 입니다.")


if all_pig_number < 1000:
    result_list = breeding_price_latest_df[:1].values[0]
    print_cost(result_list)
elif all_pig_number < 2000:
    result_list = breeding_price_latest_df[1:2].values[0]
    print_cost(result_list)
elif all_pig_number < 3000:
    result_list = breeding_price_latest_df[2:3].values[0]
    print_cost(result_list)
elif 3000 < all_pig_number:
    result_list = breeding_price_latest_df[3:4].values[0]
    print_cost(result_list)
else:
    result_list = breeding_price_latest_df[4:5].values[0]
    print_cost(result_list)

전기세수도광열비 : 9934100원 
농구비 : 11643500원 
영농시설비 : 10681000원 
기타재료비 : 1859000원 
차입금이자 : 6887100원 
토지임차료 : 410300원 
고용노동비 : 13517900원 
분뇨처리비 : 13772000원 
생산관리비 : 1786400원 
기타비용 : 1613700원 
------------------------------------
예상되는 총 1년 운영비용 : 72105000원 입니다.
예상되는 월 운영비용 : 6008750원 입니다.


In [87]:
breeding_price_latest_df[:1].values[0][0]

8375